In [1]:
# 자연어 처리나 음성 처리분야에서 많이 사용되는 RNN의 기본적인 사용법
# 4개의 글자를 가진 단어를 학습시켜, 3글자만 주어지면 나머지 한 글자를 추천하여 단어를 완성
import tensorflow as tf
import numpy as np

char_arr = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']

In [2]:
# one-hot 인코딩 사용 및 디코딩을 위한 연관 배열 생성
# {'a' : 0, 'b': 1, ...}
num_dic = {n : i for i,n in enumerate(char_arr)}
dic_len = len(num_dic)

# 다음 배열은 입력값과 출력값으로 다음처럼 사용
# wor -> X, d-> Y
# woo -> X, d-> Y
seq_data = ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love','kiss', 'kind']

In [3]:
def make_batch(seq_data):
    input_batch = []
    target_batch = []
    
    for seq in seq_data:
        # 여기서 생성하는 input_batch 와 target_batch는 
        # 알파벳 배열의 인덱스 번호
        # [22, 14, 17] [22, 14,14] [3,4,4] [3,8,21]
        
        input = [num_dic[n] for n in seq[:-1]]
        # 3, 3, 15, 4, 3...
        target = num_dic[seq[-1]]
        # one-hot 인코딩
        input_batch.append(np.eye(dic_len)[input])
        # 지금까지 손실함수로 사용하던 softmax_cross_entropy_with_logits 함수는 label 값을 one-hot 인코딩으로 넘겨야 하지만,
        # sparse_softmax_cross_entropy_with_logits 는 one-hot 인코딩을 사용하지 않으므로 index를 넘겨줌
        target_batch.append(target)
        
    return input_batch, target_batch

In [4]:
#######
# 옵션 설정
#######
learning_rate = 0.01
n_hidden = 128
total_epoch = 30
# 타입 스텝 : [1 2 3] => 3
# RNN을 구성하는 시퀀스의 개수
n_step = 3
# 입력값 크기. 알파벳에 대한 one-hot 인코딩 이므로 26개가 됨
# 출력값도 입력값과 마찬가지로 26개의 알파벳으로 분류
n_input = n_class = dic_len

In [7]:
#######
# 신경망 모델 구성
#######
X = tf.placeholder(tf.float32, [None, n_step, n_input])
# 비용함수에 sparse_softmax_cross_entropy_with_logits를 사용하므로
# 출력값과의 계산을 위한 원본값의 형태는 one-hot vector가 아니라 인덱스 숫자를 그대로 사용
# 하나의 값만 있는 1차원 배열을 받음
Y = tf.placeholder(tf.int32, [None])

W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

# RNN 셀 생성
cell1 = tf.contrib.rnn.BasicLSTMCell(n_hidden)
# 과적합 방지를 위한 dropout 기법 사용
cell1 = tf.contrib.rnn.DropoutWrapper(cell1, output_keep_prob=0.5)
# 여러개의 셀을 조합해서 사용하기 위해 셀을 추가로 생성
cell2 = tf.contrib.rnn.BasicLSTMCell(n_hidden)
# 여러개의 셀을 조합한 RNN 셀 생성
multi_cell = tf.contrib.rnn.MultiRNNCell([cell1, cell2])

outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)

# 최종결과는 one-ht 인코딩 형식으로 만듦
outputs = tf.transpose(outputs, [1, 0 , 2])
outputs = outputs[-1]
model = tf.matmul(outputs, W)+b

cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=Y))

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [8]:
#######
# 신경망 모델 학습
#######
sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, target_batch = make_batch(seq_data)

for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost], feed_dict= {X: input_batch, Y: target_batch})
    
    print('Epoch : ', '%04d' %(epoch+1), 'cost = ', '{:.6f}'.format(loss))
    
print('최적화 완료')

Epoch :  0001 cost =  3.214425
Epoch :  0002 cost =  2.399841
Epoch :  0003 cost =  1.716219
Epoch :  0004 cost =  1.002727
Epoch :  0005 cost =  1.121729
Epoch :  0006 cost =  0.479319
Epoch :  0007 cost =  0.548894
Epoch :  0008 cost =  0.514675
Epoch :  0009 cost =  0.321753
Epoch :  0010 cost =  0.387402
Epoch :  0011 cost =  0.532843
Epoch :  0012 cost =  0.431578
Epoch :  0013 cost =  0.127435
Epoch :  0014 cost =  0.205327
Epoch :  0015 cost =  0.168700
Epoch :  0016 cost =  0.294759
Epoch :  0017 cost =  0.214743
Epoch :  0018 cost =  0.147290
Epoch :  0019 cost =  0.131968
Epoch :  0020 cost =  0.141693
Epoch :  0021 cost =  0.066727
Epoch :  0022 cost =  0.095354
Epoch :  0023 cost =  0.168163
Epoch :  0024 cost =  0.042559
Epoch :  0025 cost =  0.087829
Epoch :  0026 cost =  0.036181
Epoch :  0027 cost =  0.030574
Epoch :  0028 cost =  0.074512
Epoch :  0029 cost =  0.016474
Epoch :  0030 cost =  0.012192
최적화 완료


In [9]:
#######
# 결과 확인
#######
# 레이블 값이 정수이므로 예측값도 정수로 변경
prediction = tf.cast(tf.argmax(model,1), tf.int32)
# one-hot 인코딩이 아니므로 입력값을 그대로 비교
prediction_check = tf.equal(prediction, Y)
accuracy = tf.reduce_mean(tf.cast(prediction_check, tf.float32))

input_batch, target_batch = make_batch(seq_data)

predict, accuracy_val = sess.run([prediction, accuracy], feed_dict = {X: input_batch, Y: target_batch})

predict_words = []
for idx, val in enumerate(seq_data):
    last_char = char_arr[predict[idx]]
    predict_words.append(val[:3] + last_char)

print("\n=== 예측 결과 ===")
print('입력값 : ', [w[:3] + ' ' for w in seq_data])
print('예측값 : ', predict_words)
print('정확도 : ', accuracy_val)


=== 예측 결과 ===
입력값 :  ['wor ', 'woo ', 'dee ', 'div ', 'col ', 'coo ', 'loa ', 'lov ', 'kis ', 'kin ']
예측값 :  ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']
정확도 :  1.0
